In [1]:
import os
import sys
import numpy as np
import nibabel as nib
import pickle
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline
os.chdir('/local/home/dhaziza/entrack')
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
sys.path.append('/local/home/dhaziza/entrack/')

from src.data.providers.py_streaming import get_train_test_filenames

config = {
    'classes': 2,
    'data_paths': {
        'datadir': '/local/ADNI_AIBL/ADNI_AIBL_T1_normalized/train_NC_AD/',
        'class_labels': '/local/ADNI_AIBL/ADNI_AIBL_T1_normalized/py2/AIBL_ADNI_class_labels_T1_NC_AD.pkl',
        'valid_data': '/local/ADNI_AIBL/ADNI_AIBL_T1_normalized/py2/AIBL_ADNI_valid_T1_NC_AD.pkl',
        'train_data': '/local/ADNI_AIBL/ADNI_AIBL_T1_normalized/py2/AIBL_ADNI_train_T1_NC_AD.pkl',
        'regex': '_normalized\.nii\.gz',
        'split_on': '_normalized.nii.gz',
    }
}
train, valid = get_train_test_filenames(config)

('Validation patients count in Dict: ', 800, 'Train patients count in Dict:', 14370)
('Train Class ', 0, 6218)
('Valid Class ', 0, 400)
('Train Class ', 1, 8152)
('Valid Class ', 1, 400)


In [2]:
ROOT_DATA_DIR = 'notebooks/2d_crops'
def dump_files(files, _class, train, plot=False):
    save_folder = '%s/c%s' % (ROOT_DATA_DIR, _class)
    total_todo = len(files)
    print_every = total_todo / 10
    train_text = 'train' if train else 'valid'
    files.sort()
    for i, f in enumerate(files):
        save_to = '%s/%d_%s.jpg' % (save_folder, i, train_text)
        data = nib.load(f).get_data()
        data = data[:, :, 45]
        data = (data*10 + 128)
        data = data.astype(np.uint8)
        if plot:
            print data.shape
            plt.imshow(data)
            plt.show()
        im = Image.fromarray(data)
        im.save(save_to)
        if i % print_every == 0:
            print('%s[%s]: Done %d / %d' % (save_folder, train_text, i, total_todo))
        
dump_files(train[0], 0, True)
dump_files(train[1], 1, True)
dump_files(valid[0], 0, False)
dump_files(valid[1], 1, False)

notebooks/2d_crops/c0[train]: Done 0 / 6218
notebooks/2d_crops/c0[train]: Done 621 / 6218
notebooks/2d_crops/c0[train]: Done 1242 / 6218
notebooks/2d_crops/c0[train]: Done 1863 / 6218
notebooks/2d_crops/c0[train]: Done 2484 / 6218
notebooks/2d_crops/c0[train]: Done 3105 / 6218
notebooks/2d_crops/c0[train]: Done 3726 / 6218
notebooks/2d_crops/c0[train]: Done 4347 / 6218
notebooks/2d_crops/c0[train]: Done 4968 / 6218
notebooks/2d_crops/c0[train]: Done 5589 / 6218
notebooks/2d_crops/c0[train]: Done 6210 / 6218
notebooks/2d_crops/c1[train]: Done 0 / 8152
notebooks/2d_crops/c1[train]: Done 815 / 8152
notebooks/2d_crops/c1[train]: Done 1630 / 8152
notebooks/2d_crops/c1[train]: Done 2445 / 8152
notebooks/2d_crops/c1[train]: Done 3260 / 8152
notebooks/2d_crops/c1[train]: Done 4075 / 8152
notebooks/2d_crops/c1[train]: Done 4890 / 8152
notebooks/2d_crops/c1[train]: Done 5705 / 8152
notebooks/2d_crops/c1[train]: Done 6520 / 8152
notebooks/2d_crops/c1[train]: Done 7335 / 8152
notebooks/2d_crops/c1